In [81]:
import seaborn.objects
from jax import lax
from numpy import acos
import wandb
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np
from pprint import pprint
# Initialize a wandb API object
api = wandb.Api()



In [82]:
# import wandb
# api = wandb.Api()
# run = api.run("/INR_NTK/images/runs/fhluur8f")
# print(run.history())
project = "images200k"

In [83]:
runs = api.runs(f"INR_NTK/{project}")
# runs = api.runs("INR_NTK/images/fhluur8f")

In [84]:
configs = []
history = []

In [85]:
for run in tqdm(runs):
    # pprint(run.config)
    # hst =run.history()
    cfg = run.config
    hst = run.history()
    if not hst.empty:
        configs.append(cfg)
        history.append(hst)

100%|██████████| 441/441 [03:21<00:00,  2.19it/s]


In [96]:
configs[:10]

[{'steps': {'value': 100000},
  'optimizer': {'value': 'adam'},
  'data_index': {'value': None},
  'model_type': {'value': 'inr_modules.CombinedINR'},
  'sampler__0': {'value': 'sampling.GridSubsetSampler'},
  'compute_ntk': {'value': True},
  'method_name': {'value': 'Siren_image'},
  'wandb_group': {'value': '1-Siren_image'},
  'architecture': {'value': './model_components'},
  'trainer_type': {'value': 'training.train_inr_scan'},
  'wandb_entity': {'value': 'INR_NTK'},
  'entropy_class': {'value': 1},
  'loss_function': {'value': 'losses.scaled_mse_loss'},
  'metrics__0__0': {'value': 'metrics.MSEOnFixedGrid'},
  'wandb_project': {'value': 'images'},
  'loss_evaluator': {'value': 'losses.PointWiseLossEvaluator'},
  'trainer_module': {'value': './inr_utils/'},
  'optimizer_state': {'value': None},
  'sampler__1__min': {'value': -1},
  'target_function': {'value': 'images.ContinuousImage'},
  'components_module': {'value': './inr_utils/'},
  'storage_directory': {'value': 'factory_res

In [86]:
def extract_layer_type(config_dict):
    """
    Extracts the layer type from the given configuration dictionary.

    Args:
    config_dict (dict): The configuration dictionary.

    Returns:
    str: The layer type if found, otherwise None.
    """
    layer_type_key = 'model_config__terms__0__1__layer_type'
    if layer_type_key in config_dict:
        return config_dict[layer_type_key]['value']
    return None

def extract_w0(config_dict):
    """
    Extracts w0 from the given configuration dictionary.

    Args:
    config_dict (dict): The configuration dictionary.

    Returns:
    str: The layer type if found, otherwise None.
    """
    layer_type_key = 'model_config__terms__0__1__activation_kwargs__w0'
    if layer_type_key in config_dict:
        return config_dict[layer_type_key]['value']
    return None

def extract_s0(config_dict):
    """
    Extracts s0 from the given configuration dictionary.

    Args:
    config_dict (dict): The configuration dictionary.

    Returns:
    str: The layer type if found, otherwise None.
    """
    layer_type_key = 'model_config__terms__0__1__activation_kwargs__s0'
    if layer_type_key in config_dict:
        return config_dict[layer_type_key]['value']
    return None

def extract_a(config_dict):
    """
    Extracts s0 from the given configuration dictionary.

    Args:
    config_dict (dict): The configuration dictionary.

    Returns:
    str: The layer type if found, otherwise None.
    """
    layer_type_key = 'model_config__terms__0__1__activation_kwargs__a'
    if layer_type_key in config_dict:
        return config_dict[layer_type_key]['value']
    return None

def extract_entropy(config_dict):
    """
    Extracts w0 from the given configuration dictionary.

    Args:
    config_dict (dict): The configuration dictionary.

    Returns:
    str: The layer type if found, otherwise None.
    """
    layer_type_key = 'entropy_class'
    if layer_type_key in config_dict:
        return config_dict[layer_type_key]['value']
    return None



layers = list(map(lambda x: extract_layer_type(x), configs))
w0s = list(map(lambda x: extract_w0(x), configs))
s0s = list(map(lambda x: extract_s0(x), configs))
a0s = list(map(lambda x: extract_a(x), configs))
entropys = list(map(lambda x: extract_entropy(x), configs))




In [87]:
history[0]

,ntk_condition_number,_step,lin_measure,SSIM,ntk_plot,config,_timestamp,image_on_grid,MSE_on_fixed_grid,PSNR,final_loss,_runtime,loss_plot,losses
0,4.982636,0,0.636844,NaN,"{'height': 1000, 'width': 1000, '_type': 'imag...",None,1.739792e+09,None,NaN,NaN,NaN,45.932729,None,None
1,NaN,1,NaN,NaN,None,None,1.739792e+09,None,NaN,NaN,NaN,181.145020,None,None
2,NaN,2,NaN,NaN,None,"{'architecture': './model_components',\n 'comp...",1.739792e+09,None,NaN,NaN,NaN,181.146205,None,None
3,NaN,3,NaN,0.16009,None,None,1.739792e+09,"{'height': 1356, 'width': 2040, '_type': 'imag...",3018.980469,-34.798599,0.100125,191.902692,"{'_type': 'plotly-file', 'sha256': '3ec9240bc9...","{'_type': 'histogram', 'values': [87083, 4251,..."


In [88]:
def extract_metrics(df):
    """
    Extracts the ntk_condition_number, lin_measure, SSIM, MSE_on_fixed_grid, and psnr from the dataframe.

    Args:
    df (pd.DataFrame): The input dataframe.
    layer_type: str

    Returns:
    pd.DataFrame: A dataframe with the extracted metrics.
    """
    # Define the columns to extract
    columns_to_extract = ['ntk_condition_number', 'lin_measure', 'SSIM', 'MSE_on_fixed_grid', 'PSNR']

    # Extract the relevant columns

    dct = {}
    for column in columns_to_extract:
        val = df[column].dropna().values
        dct[column] = val
    return pd.DataFrame(dct)

metrics = list(map(lambda x: extract_metrics(x), history))

data = pd.concat(metrics)

data["w0s"] =w0s
data["s0s"] =s0s
data["a0s"] =a0s
data["entropy"] = entropys

In [89]:
data.head()

,ntk_condition_number,lin_measure,SSIM,MSE_on_fixed_grid,PSNR,w0s,s0s,a0s,entropy
0,4.982636,0.636844,0.160090,3018.980469,-34.798599,27.223333,NaN,NaN,1
0,11.461319,0.398994,0.273992,1934.841797,-32.866455,NaN,NaN,40.276667,5
0,4.668180,0.395132,0.346043,3629.763428,-35.598778,27.223333,NaN,NaN,4
0,8.333210,0.349672,0.078666,2587.439209,-34.128700,NaN,NaN,33.330000,1
0,4.464695,0.024208,0.396087,1150.045288,-30.607147,17.223333,17.223333,NaN,5


In [90]:
data.to_csv(f"{project}_analysis.csv", index=False)

In [91]:
from sklearn.model_selection import train_test_split

import statsmodels.api as sm



In [99]:
Xs =data[["ntk_condition_number", "lin_measure"]]

Ys = data[["SSIM", "MSE_on_fixed_grid", "PSNR"]]



In [136]:
X = (data[["lin_measure"]] ) # Predictor (as DataFrame)
y = data["SSIM"]                     # Target (as Series)

# Add intercept term to X
X_with_const = sm.add_constant(X)

# Fit the model with CORRECT order: y first, X_with_const second
mod = sm.OLS(y, X_with_const)
res = mod.fit()
# print(res.summary())


def fit_ols(endogenous, exogenous):
    """
    Fits an OLS model to the given data.

    Args:
    endogenous (pd.Series): The endogenous variable.
    exogenous (pd.DataFrame): The exogenous variables.

    Returns:
    statsmodels.regression.linear_model.RegressionResultsWrapper: The fitted model.
    """
    # Add intercept term to X
    exogenous_with_const = sm.add_constant(exogenous)

    # Fit the model with CORRECT order: y first, X_with_const second
    mod = sm.OLS(endogenous, exogenous_with_const)
    res = mod.fit()
    return res

results = []
for x in Xs:
    for y in Ys:
        res = fit_ols(data[y], data[x])
        dct = {
            "x": x,
            "y": y,
            # "res": res,
            "fvalue": res.fvalue,
            "fpvalue": res.f_pvalue,
            "rsquared": res.rsquared,
            "rsquared_adj": res.rsquared_adj,
            # "t_vals": res.tvalues[x],
            # "p_vals": res.pvalues[x],
            # "params": res.params[x],
            # "std_err": res.bse[x],


        }
        results.append(dct)

reg_analysis = pd.DataFrame(results)








In [137]:
reg_analysis

,x,y,fvalue,fpvalue,rsquared,rsquared_adj
0,ntk_condition_number,SSIM,36.374480,4.059564e-09,0.092233,0.089698
1,ntk_condition_number,MSE_on_fixed_grid,58.769744,1.685189e-13,0.141013,0.138613
2,ntk_condition_number,PSNR,17.538881,3.548121e-05,0.046703,0.044040
3,lin_measure,SSIM,39.250393,1.070206e-09,0.098805,0.096288
4,lin_measure,MSE_on_fixed_grid,127.229844,1.872846e-25,0.262205,0.260144
5,lin_measure,PSNR,38.340839,1.629387e-09,0.096737,0.094214


In [145]:
# Create the pivot table
pivot_table = reg_analysis.pivot(index='x', columns='y', values=['fvalue', 'fpvalue', 'rsquared', 'rsquared_adj'])

# Display the pivot table
values = ['fvalue', 'fpvalue', 'rsquared', 'rsquared_adj']

for value in values:
    print(f"Value: {value}")
    print(pivot_table[value].round(3).to_latex())
    print("\n\n\n")

Value: fvalue
\begin{tabular}{lrrr}
\toprule
y & MSE_on_fixed_grid & PSNR & SSIM \\
x &  &  &  \\
\midrule
lin_measure & 127.230000 & 38.341000 & 39.250000 \\
ntk_condition_number & 58.770000 & 17.539000 & 36.374000 \\
\bottomrule
\end{tabular}





Value: fpvalue
\begin{tabular}{lrrr}
\toprule
y & MSE_on_fixed_grid & PSNR & SSIM \\
x &  &  &  \\
\midrule
lin_measure & 0.000000 & 0.000000 & 0.000000 \\
ntk_condition_number & 0.000000 & 0.000000 & 0.000000 \\
\bottomrule
\end{tabular}





Value: rsquared
\begin{tabular}{lrrr}
\toprule
y & MSE_on_fixed_grid & PSNR & SSIM \\
x &  &  &  \\
\midrule
lin_measure & 0.262000 & 0.097000 & 0.099000 \\
ntk_condition_number & 0.141000 & 0.047000 & 0.092000 \\
\bottomrule
\end{tabular}





Value: rsquared_adj
\begin{tabular}{lrrr}
\toprule
y & MSE_on_fixed_grid & PSNR & SSIM \\
x &  &  &  \\
\midrule
lin_measure & 0.260000 & 0.094000 & 0.096000 \\
ntk_condition_number & 0.139000 & 0.044000 & 0.090000 \\
\bottomrule
\end{tabular}





